# Naive Bayes Spam Filter

Code largely based on Joel Grus' *Data Science from Scratch*, page 168 (O'Reilly)

## Classifier Creation

In [152]:
from collections import defaultdict
import math
import re

class NaiveBayesSpamFilter():
   
    # Pipeline
    def train(self, training_set, k=0.1):
        """Train classifier with training_set of form (String MESSAGE, Boolean ISSPAM)"""
        # Count spam and non-spam messages
        num_spam = sum([1 if is_spam else 0 for message, is_spam in training_set])
        num_not_spam = len(training_set) - num_spam
        # Run data through our pipeline
        word_counts = self.count_words(training_set)
        self.word_probs = self.word_probabilities(word_counts, num_spam, num_not_spam, k)
    def classify(self, message):
        try:
            return self.spam_probability(self.word_probs, message)
        except:
            print("You must train first!")
    
    #Functions for pipeline
    def tokenize(self,message):
        """Return unique words in message"""
        return set(re.findall("[a-z0-9']+", message.lower()))
    def count_words(self,training_set):
        """Returns defaultdict of form {WORD:[SPAM_COUNT,NON_SPAM_COUNT]}. training_set consists of tuples (MESSAGE,IS_SPAM)"""
        counts = defaultdict(lambda : [0,0]) # Associate each word with a spam count and non-spam count
        for message, is_spam in training_set:
            for word in self.tokenize(message):
                counts[word][0 if is_spam else 1] += 1
        return counts
    def word_probabilities(self, count, total_spams, total_non_spams, k):
        """Return list of tuples (word, P(W|Spam), P(W|~Spam))"""
        word_probs = []
        for w, (spam, non_spam) in count.iteritems():
            p_word_given_spam = (spam + k) / (total_spams + 2*k) # Use of 'k' is to prevent any 0's in probabilities
            p_word_given_non_spam = (non_spam + k) / (total_non_spams + 2*k)
            word_probs.append((w,p_word_given_spam,p_word_given_non_spam))
        return word_probs
    def spam_probability(self,word_probs, message):
        """List of word probability tuples and one message, return P(Spam|all_word_in_message)"""
        message_words = self.tokenize(message)
        log_prob_given_spam = log_prob_given_not_spam = 0.0
        for word, prob_given_spam, prob_given_not_spam in word_probs:
            if word in message_words:
                # Since we are using log probabilities, we add rather than multiply the log of each word's prob
                # i.e., p(all_word|spam) = p(w1|spam)*p(w2|spam)*... = exp(log(p(w1|spam)) + log(p(w2|spam)) + ...)
                log_prob_given_spam += math.log(prob_given_spam)
                log_prob_given_not_spam += math.log(prob_given_not_spam)
            else:
                # If we don't see one of our training words in the message,
                # add the corresponding probability for not seeing it
                # i.e. log(1-probability)
                log_prob_given_spam += math.log(1.0-prob_given_spam)
                log_prob_given_not_spam += math.log(1.0-prob_given_not_spam)
        prob_given_spam = math.exp(log_prob_given_spam)
        prob_given_not_spam = math.exp(log_prob_given_not_spam)
        # Bayes thm simplified with the (naive) assumption P(S) = P(~S)
        # Bayes thm: P(A|B) = P(B|A)P(B) / (P(A|B)P(B) + P(A|~B)P(~B))
        return prob_given_spam / (prob_given_spam + prob_given_not_spam) 
        

 Now let's test it with a simple example

In [153]:
# 3 spam messages, 2 non-spam messages
message_samples = [("Hello, this is an ordinary message", False), ("free viagra!", True), 
                   ("FREE rolex", True), ("FREE money!", True), ("Happy holidays", False)]
# Create and train
spamfilter = NaiveBayesSpamFilter()
spamfilter.train(message_samples)

# The examples we try to classify
sample_spam_message = "Click the link for free viagra and a rolex"
sample_non_spam_message = "Hello friend!"

print("Probability our message '{0}' is spam: {1}".format(sample_spam_message, spamfilter.classify(sample_spam_message)))
print("Probability our message '{0}' is spam: {1}".format(sample_non_spam_message, spamfilter.classify(sample_non_spam_message)))

Probability our message 'Click the link for free viagra and a rolex' is spam: 0.999993990673
Probability our message 'Hello friend!' is spam: 0.0637988267127


## Evaluating our model

Now let's test on a larger dataset of emails from *http://spamassassin.apache.org/publiccorpus/* 

We will train and classify emails based only on the *subject*, using it as the "message"

In [154]:
# To rerun make sure '20021010_easy_ham.tar.bz2' and '20021010_spam.tar.bz2' are in your relative path
import bz2
import re

non_spam = bz2.BZ2File('20021010_easy_ham.tar.bz2').readlines()
spam = bz2.BZ2File('20021010_spam.tar.bz2').readlines()

all_messages = []
for line in non_spam:
    if line.startswith("Subject:"):
        message = re.sub(r"^Subject: ", "", line).strip()
        all_messages.append((message,False))
for line in spam:
    if line.startswith("Subject:"):
        message = re.sub(r"^Subject: ", "", line).strip()
        all_messages.append((message,True))

Now we have all our messages in the form [(String MESSAGE, Boolean ISSPAM),...] as required.

Let's take a brief look at the data

In [155]:
total_messages = len(all_messages)
total_spam = len([message for message, isspam in all_messages if isspam == True])
total_non_spam = total_messages - total_spam

print("Total spam messages: " + str(total_spam))
print("A spam message:" + all_messages[-1][0])
print("\nTotal non-spam messages: " + str(total_non_spam))
print("A non-spam message:" + all_messages[0][0])
print("\nTotal examples: " + str(total_messages))

Total spam messages: 503
A spam message:Want to play poker with other people online.

Total non-spam messages: 2651
A non-spam message:RE: The Curse of India's Socialism

Total examples: 3154



Next, we shuffle and split up our data into training (75%) and testing (25%) sets

In [156]:
import random

random.shuffle(all_messages)
training_set = all_messages[:2300]
testing_set = all_messages[2300:]

### Model training & testing

In [157]:
spamfilter = NaiveBayesSpamFilter()
spamfilter.train(training_set)

total_correct = 0.0 # Tally up correctly classified emails
mis_labeled = [] # To hold mis-labeled emails

for message, isspam in testing_set:
    p_spam = spamfilter.classify(message)
    label = (p_spam > .5)
    if(label == isspam):
        total_correct+=1
    else:
        mis_labeled.append((message,isspam, p_spam))
        
print("Model accuracy: {}".format(total_correct/len(testing_set)))

print("\nSome mis-labeled emails:\n")
print("Real Label\tP(S)\tMessage")
for i in range(5):
    label = ("Spam:\t\t" if mis_labeled[i][1] else "Non-Spam:\t")
    print(label + str(round(mis_labeled[i][2],3)) + "\t" + mis_labeled[i][0])

Model accuracy: 0.909836065574

Some mis-labeled emails:

Real Label	P(S)	Message
Spam:		0.373	[scoop] Haberdar olun
Non-Spam:	0.655	Turning junk computers into activist gold
Spam:		0.034	re: domain registration savings
Spam:		0.472	Hottest babes!                                                       4139YAEk3--9
Spam:		0.042	[scoop] ....It is not my fault. .- vwiid
